# Traditional methods - OpenCV+scikit_learn

It's for traditional methods of object detection, using OpenCV to preprocess and extract features and then use machine learning algorithm to classify. 
Generally, it can be divided into three modules: preprocessing, feature extraction and classification.
First, some prepration works.

In [3]:
import cv2
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from lib.data_utils import get_MNIST_data

Then read the MNIST data.

In [2]:
data = get_MNIST_data()

# check if we load the data successfully
print(data['X_train'].shape)

(41000, 28, 28, 1)


## Preprocessing

# Feature extraction

Different methods exist to extract feature. Here we try ORB (Oriented FAST and Rotated BRIEF).

In [ ]:
# initiate and configure ORB
orb = cv2.ORB(nfeatures=50)

# compute the ORB descriptors
feat = {}
for key in ['X_train', 'X_test']:
    _, feat[key] = orb.detectAndCompute(data[key])

# Classification

Different machine learning methods can be used to classify the digits. First we try decision tree.

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(feat['X_train'],data['y_train'])